<center>
    
# R406: Applied Economic Modelling with Python

</center>

<br> <br> 

<center>

## Symbolic computing with SymPy

</center>

<br><br> 

<center>
<b> Andrey Vassilev </b>
</center>



# Outline

1. Introduction
2. Basic SymPy objects and operations
3. Applications of specific modules
    - calculus - limits, derivatives, integrals
    - linear algebra
    - solving equations
    - statistics

# Introduction: what is SymPy?

- SymPy is a Python library for computer algebra/symbolic computing.
- Roughly speaking, this means that you can use it to perform computations in a pen-and-paper manner, getting symbolic expressions rather that numbers as a result.
- SymPy is similar to other CAS like Mathematica, Maple, Matlab's Symbolic Math Toolbox or Maxima.
- SymPy is designed to be equally usable interactively or programmatically as a part of a larger workflow.
- See [this paper](https://peerj.com/articles/cs-103/) or the official [documentation](http://docs.sympy.org/latest/index.html) for more information.

# SymPy basics

Here is a simple example to illustrate the difference between "regular" computations and symbolic ones.

In a standard use case, we define variables and perform manipulations on them to get numerically evaluated results.

In [ ]:
x = 2
y = 5
z = x**2 - y**3
z

Notice that the third assignment, `z = x**2 - y**3`, binds the variable `z` to the specific, evaluated result, -121 in this case. There is no idea of keeping the expression $x^2-y^3$ as such.

Here is how the same operations work in symbolic form:

In [ ]:
import sympy as sym
x,y = sym.symbols('x y')
z = x**2 - y**3
z

The `symbols` function does just that — create Symbol objects `x` and `y` holding the symbols "x" and "y". There is nothing to prevent you from defining the objects to represent different symbols but it is confusing and discouraged by convention:

In [ ]:
a,b = sym.symbols("X, Y") # Notice we can separate with commas as well
print(a**2+b**4)
del a,b

If you are disciplined enough to conform to the above convention, you can make use of the `var()` function, which creates Symbol objects with the same name as the arguments:

In [ ]:
sym.var("x1 x2 x3")
x1

Obviously, more complicated expressions can be constructed but these can be difficult to read:

In [ ]:
z = (sym.sqrt(x)+sym.exp(y**2)*sym.cos(x+sym.pi))/(x**(sym.ln(y))+y**5)
# Note that SymPy offers symbolic versions of different functions
# like sqrt, exp etc.
z

For that reason, SymPy offers pretty-printing functionality using available engines such as LaTeX or Matplotlib.

In [ ]:
sym.init_printing(use_unicode=True)
z

For the LaTeX fans out there, you can also access the $\LaTeX$ code representing an expression by using the `latex()` function:

In [ ]:
print(sym.latex(z))

SymPy is clever enough to try to simplify expressions when you combine them:

In [ ]:
z1 = x**2 - 4*x*y - 5*y**3
z2 = 3*x**2 + x*y + 4*y**3

In [ ]:
z2+z1

Note that if you want to pretty print several objects in the same cell, you need to import `display` from `IPython.display`.

In [ ]:
from IPython.display import display
display(z1)
display(z2)
display(z1+z2)

Symbols can also be created in "packaged" form, i.e. placed in a suitable container:

In [ ]:
SymList = sym.symbols(['a', 'b', 'c', 'd'])
display(SymList)
display(SymList[1])

In [ ]:
SymSet = sym.symbols({'a', 'b', 'c', 'd'})
display(SymSet)
sym.symbols('a') in SymSet

If you need to create many symbols, there is range syntax to reduce typing:

In [ ]:
ST = sym.symbols("x1:10")
ST

In [ ]:
ST[0]**2+sym.exp(ST[3])

In [ ]:
ST = sym.symbols("a:g")
ST

In [ ]:
# Don't forget to clean up before you leave
del SymList, SymSet, ST

Let's see what we can do with symbols and symbolic expressions.

# Substitution

It is possible to perform substitutions using the `subs()` method:

In [ ]:
display(z)
z.subs(y,x1**2-3)

We can also substitute specific numeric values:

In [ ]:
z.subs(x,1)
# Note that cos(1) below is not evaluated

In [ ]:
z.subs({x:1,y:0}) # Equivalently: z.subs([(x,1),(y,0)])

# Evaluation

Notice that SymPy will simplify expressions when it can but will not evaluate in general: in the preceding example `cos(1)` remained unevaluated. We can require evaluation to a floating point number by using the `evalf()` method. 

In [ ]:
sym.cos(1)

In [ ]:
sym.cos(1).evalf()

In [ ]:
type(sym.cos(1).evalf()) # The type is a special SymPy float

In [ ]:
type(float(sym.cos(1).evalf())) # But it is convertible to a regular float

Some more examples:

In [ ]:
sym.cos(sym.pi/2) # Note the symbolic constant pi

In [ ]:
type(sym.cos(sym.pi/2))

In [ ]:
sym.sqrt(9)

In [ ]:
sym.sqrt(5)

In [ ]:
sym.sqrt(5).evalf()

# From strings to expressions

A string can be converted into an expression using the `sympify()` function, provided that you are willing to risk an `eval()` call, which is performed internally. 

**Note the name of the function: it's `sympify` = "make a SymPy object", NOT `simplify` = "make simpler", which is another SymPy function!**

In [ ]:
sym.sympify("a**2 + a*b + b**3")

# Evaluating at many points

Sometimes you may want to take an expression and evaluate it for an entire array of values. This can be done efficiently with the `lambdify()` function. As the name hints, `lambdify()` produces a lambda function that is usable with a numerical package, usually NumPy.

In [ ]:
import numpy as np
a = np.arange(-5,5)

In [ ]:
expr = z2.subs(y,sym.exp(x))
expr

In [ ]:
# Produce a NumPy-compatible lambda function from expression expr,
# treating the expression as a function of x
f = sym.lambdify(x,expr,modules="numpy") 
f(a)

# Simplification

This is performed by the `simplify()` function. Here it is in action:

In [ ]:
expr = (x**2-x-6)/(x-3)
display(expr)
sym.simplify(expr)

In [ ]:
sym.simplify( sym.factorial(x)/sym.factorial(x-2) )

- While simplification is intuitive and obvious in simple cases, there is no universal, mathematically rigorous way to do it in general.
- As a result, `simplify()` is implemented to try various heuristics to make an expression "simple" according to its internal criteria but:
    - it can be slow;
    - it is not guaranteed to deliver a result;
    - the result may not be what you expect or what can be obtained by pen-and-paper calculations.

- SimPy has more specialized simplification routines to handle particular cases.
- Thus, you are advised to use `simplify()` in cases where you don't exactly know what is happening and are trying to clean up an expression a little.

Two specialized types of simplification routines are given by the `factor()` and `expand()` functions.

In [ ]:
sym.factor( x**3-3*x**2-33*x+35 )

In [ ]:
sym.expand( (x-5)**3 )

The `collect()` function groups together common powers of a variable:

In [ ]:
x,y,z = sym.symbols('x y z')
expr = x*y + x - 3 + 2*x**2 - z*x**2 + x**3
display(expr)
sym.collect(expr,x)

You can also use the `coeff()` method — whose syntax is `expr.coeff(x, n)` — to extract the coefficient in front of a particular power `n` from expression `expr`.

In [ ]:
sym.collect(expr,x).coeff(x,2)

The `cancel()` function does what its name suggests:

In [ ]:
sym.cancel((x**3 + 2*x**2 - 5*x - 6)/(x**2 - x - 2))

In many cases you can use `factor()` to achieve the same but according to the documentation `cancel()` is more efficient.

It is possible to specify the type of symbolic variable:

In [ ]:
x, y = sym.symbols('x y', positive=True)
a, b = sym.symbols('a b', real=True)

And, for example, perform power simplification operations on such variables:

In [ ]:
sym.powsimp(x**a*x**b)

# Calculus

## Limits

Limits are calculated using the `limit()` function.

In [ ]:
x, y = sym.symbols('x y')
a, b = sym.symbols('a b')

In [ ]:
sym.limit((x*y)/sym.sqrt(x),x,0)

In [ ]:
sym.limit((x*y)/sym.sqrt(x),y,5)

The special variable `oo` represents infinity:

In [ ]:
sym.limit(x/sym.sqrt(x),x,sym.oo)

You can evaluate a one-sided limit by passing `'+'` or `'-'` as another argument:

In [ ]:
sym.limit(1/x, x, 0, '+')

In [ ]:
sym.limit(1/x, x, 0, '-')

# Calculus

## Differentiation

Differentiation is performed by means of the `diff()` function.

In [ ]:
sym.diff(sym.sqrt(x),x)

You can pass either several variables or a variable and a number to compute higher-order derivatives:

In [ ]:
sym.diff(sym.sqrt(x),x,x,x)

In [ ]:
sym.diff(sym.sqrt(x),x,3)

You can also evaluate mixed derivatives in the same manner:

In [ ]:
sym.diff(sym.exp(x*sym.cos(y)),x,y)

# Calculus

## Integration

The `integrate()` function performs definite or indefinite integration. Passing an expression and a variable computes the indefinite integral (antiderivative):

In [ ]:
sym.integrate(x**3,x)

In [ ]:
sym.integrate(sym.exp(2*y)*x**3,x,y)

Notice that an integration constant is not included in the result. If you insist on getting it automatically, the [SymPy tutorial](http://docs.sympy.org/latest/tutorial/) recommends using ODE solution functionality. 

For definite integrals, pass a tuple of the variable and its limits.

In [ ]:
sym.integrate(x**2, (x, 0, 1))

In [ ]:
sym.integrate(1/sym.sqrt(2*sym.pi)*sym.exp(-x**2/2),(x,-sym.oo,sym.oo))

The same approach works for several variables:

In [ ]:
sym.integrate(sym.exp(-x**2 - y**2), 
              (x, -sym.oo, sym.oo), (y, -sym.oo, sym.oo))

# Linear algebra

## Constructing matrices

A SymPy matrix is constructed in a manner similar to a NumPy array.

In [ ]:
M = sym.Matrix([[1,5,2],
                [3,-5,8],
                [-7,7,11]])
M

A column vector is constructed as a matrix by passing an ordinary list.

In [ ]:
v = sym.Matrix([2,5,7])
v

Standard operations then follow naturally:

In [ ]:
M*v

In [ ]:
M**2

In [ ]:
M+M**2

Matrix inversion is done by raising to power -1:

In [ ]:
M**(-1)

A vector/matrix transpose is obtained just like in NumPy, by using the `T` attribute:

In [ ]:
v.T*M

Submatrices and individual elements are accessible with the usual indexing and slicing syntax:

In [ ]:
M[0,1]

In [ ]:
M[0,:]

In [ ]:
M[:,1]

In [ ]:
M[1:,1:]

The `eye()`, `ones()`, `zeros()` and `diag()` functions are also available:

In [ ]:
sym.eye(3)

In [ ]:
sym.diag(1,2,3)



# Linear algebra

## Determinants

A determinant can be computed using the `det()` method.

In [ ]:
M.det()

More interestingly, you can create a symbolic matrix and, e.g., compute its determinant.

In [ ]:
Ms = sym.Matrix(2,2,[a,b,x,y])
display(Ms)
Ms.det()



# Linear algebra

## Linear systems

A linear system can be solved by means of the `LUsolve()` method of a matrix. The method takes the right-hand side vector of the system.

In [ ]:
B = sym.Matrix([187,187,440])
X = M.LUsolve(B)
X

In [ ]:
M*X



# Linear algebra

## Eigenvectors and eigenvalues

These can be obtained using the `eigenvals()` and `eigenvects()` methods.

In [ ]:
A = sym.Matrix([[7,-4],[5,-2]])
A.eigenvals()

In the above output, the keys in the dictionary are the eigenvalues and the values are their algebraic multiplicities (i.e. they show how many times the eigenvalue is repeated).

In [ ]:
A.eigenvects() # Returns eigenvalue, algebraic multiplicity and eigenvectors

# Solving equations

## Algebraic equations

A symbolic equation in SymPy is expressed using the `Eq` function.

In [ ]:
eq = sym.Eq(2*x+4*y,-x-y**2)
eq

We can solve it using the `solveset()` function.

In [ ]:
sym.solveset(eq,x)

In [ ]:
sym.solveset(eq,y)

We can also provide an expression instead of an equality object. SymPy then assumes that the equation is of the form `expression = 0`. You can additionally instruct `solveset` to work in a specific domain (real or complex numbers).

In [ ]:
sym.solveset(x**2+1,x) # Complex numbers assumed by default

# Equivalently:
# sym.solveset(x**2+1,x,domain=sym.S.Complexes)

In [ ]:
sym.solveset(x**2+1,x,domain=sym.S.Reals)

If you need to solve a nonlinear *system* of equations, the manual recommends using `solve()`:

In [ ]:
sym.solve([x*y - 1, x - 2], x, y)

# Solving equations

## Differential equations

To solve differential equations, we create symbols that are of type *unknown function*:

In [ ]:
f, g = sym.symbols('f g', cls=sym.Function)

We can then pass arguments to these functions. They will not produce anything specific (since we have not specified a functional form) but will be recognized as functions.

In [ ]:
f(x)

In [ ]:
g(y)

Such objects can be notionally differentiated:

In [ ]:
f(x).diff(x) # Notice that diff() works as a method

In [ ]:
f(x).diff(y)

We can then use `dsolve()` to try to find a solution:

In [ ]:
sym.dsolve(f(x).diff(x)-f(x)/x,f(x))

Or, as another example:

In [ ]:
sol = sym.dsolve(f(x).diff(x,2)-6*x,f(x))
sol

And we can even solve for the constants `C1` and `C2` if we have appropriate boundary conditions:

In [ ]:
C1,C2 = sym.symbols('C1 C2')
sym.solve([sol.args[1].subs(x,0)-0,
       sol.args[1].subs(x,1)-1],C1,C2)

# Statistics

SymPy has a submodule `stats` that can help with random quantities. You are encouraged to  check out the [docs](http://docs.sympy.org/latest/modules/stats.html) but here are some illustrations to give you an idea.

## The normal distribution

In [ ]:
from sympy.stats import Normal, density, E, variance, std
mu = sym.symbols("mu")
sigma = sym.symbols("sigma", positive=True)
z = sym.symbols("z")
X = Normal("x", mu, sigma)

In [ ]:
density(X)(z)

In [ ]:
E(X)

In [ ]:
sym.simplify(E(X)) # This is required in older versions of SymPy to clean up the output

In [ ]:
variance(X) # or sym.simplify(variance(X))

In [ ]:
std(X) # or, if necessary: sym.simplify(std(X))

## The Poisson distribution

In [ ]:
from sympy.stats import Poisson
rate = sym.symbols("lambda", positive=True)
k = sym.symbols("k")
X = Poisson("x", rate)

In [ ]:
density(X)(k)

In [ ]:
E(X)

In [ ]:
variance(X) # if you get messy output, try this: sym.simplify(variance(X))